In [63]:
import pandas as pd

In [64]:
import pandas as pd
merged_data = pd.read_csv("Merged_Sales_data_v2.2.csv")

In [65]:
merged_data.columns

Index(['년분기', '행정동_코드', '행정동', '업종명', '월매출', '주중매출', '주말매출', '인구수', '점포수',
       '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수',
       '버스정류장수'],
      dtype='object')

In [66]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374581 entries, 0 to 374580
Data columns (total 17 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   년분기        374581 non-null  int64  
 1   행정동_코드     374581 non-null  int64  
 2   행정동        374581 non-null  object 
 3   업종명        374581 non-null  object 
 4   월매출        374581 non-null  int64  
 5   주중매출       374581 non-null  int64  
 6   주말매출       374581 non-null  int64  
 7   인구수        374581 non-null  int64  
 8   점포수        374581 non-null  int64  
 9   지역생활인구     374581 non-null  int64  
 10  장기외국인      374581 non-null  int64  
 11  단기외국인      374581 non-null  int64  
 12  주차장면적(면)   374581 non-null  float64
 13  주차장개수(개소)  374581 non-null  float64
 14  학교수        374581 non-null  float64
 15  학생수        374581 non-null  float64
 16  버스정류장수     374581 non-null  int64  
dtypes: float64(4), int64(11), object(2)
memory usage: 48.6+ MB


In [71]:
test_data = merged_data[merged_data["업종명"] == "한식음식점"]
test_data = test_data[['년분기', '행정동', '월매출', '주중매출', '주말매출', '인구수', '점포수',
       '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수',
       '버스정류장수']]

In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Feature와 Target 데이터 분리
# feature_data = ['년분기', '행정동', '인구수', '점포수', '지역생활인구', '장기외국인', 
#                 '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수']
feature_data = ['년분기', '행정동', '인구수', '점포수', '지역생활인구', '장기외국인', 
                '단기외국인', '주차장개수(개소)', '학교수', '학생수', '버스정류장수']
target_data = ['월매출']

# Feature와 Target 분리
X = test_data[feature_data]
y = test_data[target_data]

# Step 2: '행정동' 원-핫 인코딩
one_hot_encoded_dong = pd.get_dummies(X['행정동'], prefix='행정동')
X = pd.concat([X.drop('행정동', axis=1), one_hot_encoded_dong], axis=1)

for c in X:
    # print(X_train[c].dtype == "int64")
    if X[c].dtype == "int64":
        X[c] = X[c].astype("int16")
    elif X[c].dtype == "float64":
        X[c] = X[c].astype("float16")

# Step 3: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.dtypes)
# for c in X_train:
#     # print(X_train[c].dtype == "int64")
#     if X_train[c].dtype == "int64":
#         X_train[c] = X_train[c].astype("int16")
#     elif X_train[c].dtype == "float64":
#         X_train[c] = X_train[c].astype("float16")

# Step 4: 다항회귀 모델 준비
degree = 2  # 다항 차수
poly = PolynomialFeatures(degree=degree, include_bias=False)
scaler = StandardScaler()
model = LinearRegression()

# Step 5: 다항회귀 변환 및 학습
X_train_poly = poly.fit_transform(X_train)  # 다항 특성 생성
X_train_poly_scaled = scaler.fit_transform(X_train_poly)  # 표준화
model.fit(X_train_poly_scaled, y_train)  # 모델 학습

# Step 6: 테스트 데이터 처리 및 예측
X_test_poly = poly.transform(X_test)  # 다항 특성 생성
X_test_poly_scaled = scaler.transform(X_test_poly)  # 표준화
y_pred = model.predict(X_test_poly_scaled)  # 예측

# Step 7: 평가
train_rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train_poly_scaled)))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
train_r2 = r2_score(y_train, model.predict(X_train_poly_scaled))
test_r2 = r2_score(y_test, y_pred)

print(f"훈련 데이터 RMSE: {train_rmse:.2f}")
print(f"테스트 데이터 RMSE: {test_rmse:.2f}")
print(f"훈련 데이터 R²: {train_r2:.2f}")
print(f"테스트 데이터 R²: {test_r2:.2f}")


년분기         int16
인구수         int16
점포수         int16
지역생활인구      int16
장기외국인       int16
            ...  
행정동_효창동      bool
행정동_후암동      bool
행정동_휘경1동     bool
행정동_휘경2동     bool
행정동_흑석동      bool
Length: 433, dtype: object
훈련 데이터 RMSE: 904695521.34
테스트 데이터 RMSE: 5114576706210.87
훈련 데이터 R²: 0.99
테스트 데이터 R²: -224592.08


In [ ]:
for c in X_train:
    # print(X_train[c].dtype == "int64")
    if X_train[c].dtype == "int64":
        X_train[c] = X_train[c].astype("int16")
    elif X_train[c].dtype == "float64":
        X_train[c] = X_train[c].astype("float16")

In [ ]:
X_train.dtypes

년분기         int16
행정동_코드      int16
인구수         int16
점포수         int16
지역생활인구      int16
            ...  
행정동_효창동      bool
행정동_후암동      bool
행정동_휘경1동     bool
행정동_휘경2동     bool
행정동_흑석동      bool
Length: 435, dtype: object

In [74]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Feature와 Target 데이터 분리
feature_data = ['년분기', '행정동', '인구수', '점포수', '지역생활인구', '장기외국인', 
                '단기외국인', '주차장개수(개소)', '학교수', '학생수', '버스정류장수']
target_data = ['월매출']

# Feature와 Target 분리
X = test_data[feature_data]
y = test_data[target_data]

# Step 2: '행정동' 원-핫 인코딩
one_hot_encoded_dong = pd.get_dummies(X['행정동'], prefix='행정동')
X_numeric = X.drop('행정동', axis=1)  # 숫자형 데이터만 분리

# 데이터 타입 축소 (메모리 절약)
for c in X_numeric:
    if X_numeric[c].dtype == "int64":
        X_numeric[c] = X_numeric[c].astype("int16")
    elif X_numeric[c].dtype == "float64":
        X_numeric[c] = X_numeric[c].astype("float16")

# Step 3: Train/Test Split
X_train_num, X_test_num, y_train, y_test = train_test_split(X_numeric, y, test_size=0.2, random_state=42)
X_train_dong, X_test_dong = train_test_split(one_hot_encoded_dong, test_size=0.2, random_state=42)

# Step 4: 다항회귀 모델 준비
degree = 2  # 다항 차수
poly = PolynomialFeatures(degree=degree, include_bias=False)
scaler = StandardScaler()
model = LinearRegression()

# Step 5: 다항회귀 변환 및 학습
X_train_poly = poly.fit_transform(X_train_num)  # 다항 특성 생성
X_train_poly_scaled = scaler.fit_transform(X_train_poly)  # 표준화

# '행정동' 원-핫 인코딩 데이터와 결합
X_train_combined = np.hstack([X_train_poly_scaled, X_train_dong.to_numpy()])

# 모델 학습
model.fit(X_train_combined, y_train)

# Step 6: 테스트 데이터 처리 및 예측
X_test_poly = poly.transform(X_test_num)  # 다항 특성 생성
X_test_poly_scaled = scaler.transform(X_test_poly)  # 표준화

# '행정동' 원-핫 인코딩 데이터와 결합
X_test_combined = np.hstack([X_test_poly_scaled, X_test_dong.to_numpy()])

# 예측
y_pred = model.predict(X_test_combined)

# Step 7: 평가
train_rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train_combined)))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
train_r2 = r2_score(y_train, model.predict(X_train_combined))
test_r2 = r2_score(y_test, y_pred)

print(f"훈련 데이터 RMSE: {train_rmse:.2f}")
print(f"테스트 데이터 RMSE: {test_rmse:.2f}")
print(f"훈련 데이터 R²: {train_r2:.2f}")
print(f"테스트 데이터 R²: {test_r2:.2f}")


훈련 데이터 RMSE: 1817963556.14
테스트 데이터 RMSE: 2267159500.19
훈련 데이터 R²: 0.97
테스트 데이터 R²: 0.96


In [75]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# MAE, MSE, RMSE 계산
train_mae = mean_absolute_error(y_train, model.predict(X_train_combined))
test_mae = mean_absolute_error(y_test, y_pred)

train_mse = mean_squared_error(y_train, model.predict(X_train_combined))
test_mse = mean_squared_error(y_test, y_pred)

train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)

# 결과 출력
print(f"훈련 데이터 MAE: {train_mae:.2f}")
print(f"테스트 데이터 MAE: {test_mae:.2f}")
print(f"훈련 데이터 MSE: {train_mse:.2f}")
print(f"테스트 데이터 MSE: {test_mse:.2f}")
print(f"훈련 데이터 RMSE: {train_rmse:.2f}")
print(f"테스트 데이터 RMSE: {test_rmse:.2f}")


훈련 데이터 MAE: 1019020354.41
테스트 데이터 MAE: 1203153790.10
훈련 데이터 MSE: 3304991491451934208.00
테스트 데이터 MSE: 5140012199284261888.00
훈련 데이터 RMSE: 1817963556.14
테스트 데이터 RMSE: 2267159500.19


In [86]:
a = merged_data[merged_data["업종명"] == "한식음식점"]

print("최고 점포 수   : ", a["점포수"].max())
print("최소 점포 수   : ", a["점포수"].min())
print("평균 점포 수   : ", a["점포수"].mean())

print("최고 분기 매출 : ", a["월매출"].max())
print("최소 분기 매출 : ", a["월매출"].min())
print("평균 분기 매출 : ", a["월매출"].mean())

최고 점포 수   :  1058
최소 점포 수   :  4
평균 점포 수   :  128.78125
최고 분기 매출 :  120193529541
최소 분기 매출 :  860310
평균 분기 매출 :  7227325776.611321


In [102]:
sorted_indices = a["점포수"].sort_values(ascending=False).index

top_10_rows = a.loc[sorted_indices[1:11], :]
top_10_rows.iloc[:10,:9].groupby("년분기").max()


,행정동_코드,행정동,업종명,월매출,주중매출,주말매출,인구수,점포수
년분기,,,,,,,,
20191,11110615,종로1·2·3·4가동,한식음식점,66663258944,51368337759,15294921185,8518,1037
20192,11110615,종로1·2·3·4가동,한식음식점,67594972881,52577031124,15017941757,8500,1044
20201,11110615,종로1·2·3·4가동,한식음식점,55658290992,44826553499,10831737493,8539,1036
20202,11110615,종로1·2·3·4가동,한식음식점,56644152057,45616011199,11028140858,8526,1036
20204,11110615,종로1·2·3·4가동,한식음식점,53794810061,42794982978,10999827083,8448,1036
20213,11680640,역삼1동,한식음식점,56326836066,47444756770,8882079296,36560,1037
20214,11680640,역삼1동,한식음식점,84453117856,70957247471,13495870385,36590,1055
20221,11680640,역삼1동,한식음식점,69952384233,58275632338,11676751895,36431,1051
20223,11680640,역삼1동,한식음식점,105523677136,88528620174,16995056962,35780,1047
